In [ ]:
import ipywidgets as widgets
from ipyfilechooser import FileChooser
import os
import pandas as pd
import matplotlib.pyplot as plt
import notebook_helper
import time

%autosave 0

# controls the default starting dir for the file choosers
BASE_DIR='./output'

In [ ]:
def load_cdf(test_dir):
    cached = os.path.join(test_dir, 'analysis', 'pandas', 'cdf.gz')
    if os.path.exists(cached):
        return pd.read_pickle(cached)
    
    files = notebook_helper.find_files(test_dir, 'tracestat-cdf.txt')
    if len(files) == 0:
        raise ValueError('test dir {test_dir} does not contain tracestat-cdf.txt - try running:\n./analyze.py extract {test_dir}/test-output.tgz'.format(test_dir=test_dir))
    return notebook_helper.cdf_to_pandas(files[0])


def collect_inputs():
    test_runs = []
    for box in choosers.children:
        legend = box.children[0].value
        test_dir = box.children[1].selected_path
        if test_dir is None:
            continue
        if legend == '':
            legend = os.path.basename(test_dir)
        cdf = load_cdf(test_dir)
        test_runs.append({'legend': legend, 'test_dir': test_dir, 'cdf': cdf})
    return test_runs


def normalize_cdf(cdf):
    maxval = cdf['count'].max()
    normalized = cdf.copy()
    normalized['count'] = normalized['count'] / maxval
    return normalized


def plot_cdfs(test_runs):
    if len(test_runs) == 0:
        print('no test runs selected')
        return
    fig = plt.figure(figsize=(11,6))
    fig.suptitle("Latency CDF")
    plt.ylabel('CDF (of # of messages)')
    plt.xlabel('Message Propagation Latency (ms)')
    ax = plt.axes()
    ax.set_facecolor("white") # Setting the background color
    ax.spines['bottom'].set_color('0.5')
    ax.spines['top'].set_color('0.5')
    ax.spines['right'].set_color('0.5')
    ax.spines['left'].set_color('0.5')
    plt.rc('grid', linestyle="--", color='grey')
    plt.grid(True)
    plt.show()
    
    for r in test_runs:
        cdf = normalize_cdf(r['cdf'])
        plt.plot('delay_ms', 'count', data=cdf, label=r['legend'])
    plt.legend()
    return fig


def new_chooser():
    fc = FileChooser(BASE_DIR)
    fc.use_dir_icons = True
    name_field = widgets.Text(description='Legend Key')
    return widgets.HBox([name_field, fc])

choosers = widgets.VBox([new_chooser(), new_chooser()])

def add_chooser(btn):
    choosers.children = tuple(list(choosers.children) + [new_chooser()])

add_button = widgets.Button(description="Add another test run")
add_button.on_click(add_chooser)

output_dir_chooser = FileChooser(BASE_DIR)
output_dir_chooser.default_filename = 'merged-cdf-figures-' + time.strftime("%Y%m%d-%H%M%S")

selection_ui = widgets.VBox([
    widgets.Label("Use the Select buttons to choose test output directories."),
    widgets.Label("Set Legend Key to a friendly name to display in the legend for each result."),
    choosers,
    add_button,
    widgets.Label("Select the directory to save figure images (will be created if needed):"),
    output_dir_chooser,
])


### Select test output directories

In [ ]:
display(selection_ui)

### Once everything is selected above, run the cell below:

In [ ]:
fig = plot_cdfs(collect_inputs())

if fig is not None:
    if output_dir_chooser.selected_path is None:
        print('not saving figure images, no output dir selected')
    else:
        out_dir = os.path.join(output_dir_chooser.selected_path, output_dir_chooser.selected_filename)
        save_fig = notebook_helper.save_fig_fn(out_dir)
        save_fig(fig, 'cdf-comparison')